In [16]:
import nibabel as nib
import os
import sys
import subprocess
import glob
import random
import multiprocessing

In [17]:
# f = "/Users/brac4g/Desktop/reg4D/test.file"

In [18]:
# touch(f)

In [19]:
# touch_empty(f)

In [20]:
class File():
    '''
    File class doc-string
    
    Attributes:
        filename (string): Filename with path to file. File does not need to exist.
    '''
    
    def __init__(self,filename):
        '''
        Return filename string
        '''
        self.filename = filename
        return None
        
    def touch(filename):
        '''
        Analagous to UNIX's touch command. Creates an empty file.

        Arguments:
            filename (string): Filename (with path)

        Returns:
            filename (string): Path to file. This now file exists and can now be used as inputs to other functions.
        '''

        if not os.path.exists(filename):
            with open(filename,"w"): pass
            filename = os.path.abspath(filename)
        else:
            print(f"file: {filename} already exists. File will not be overwritten.")

        return filename
    
    def make_filename_path(filename):
        '''
        Analagous to UNIX's touch command. Creates an empty file, and then removes it - preserving the
        absolute path to the file.

        Arguments:
            filename (string): Filename (with path)

        Returns:
            filename (string): Path to file. This now file exists and can now be used as inputs to other functions.
        '''

        if not os.path.exists(filename):
            with open(filename,"w"): pass
            filename = os.path.abspath(filename)
            os.remove(filename)
        else:
            print(f"file: {filename} already exists. File will not be overwritten.")

        return filename

In [15]:
# t = File(f)

In [16]:
# t

In [18]:
# t.touch

<bound method File.touch of <__main__.File object at 0x7ff0d8cd9978>>

In [12]:
# t = File.make_filename_path(f)

In [21]:
# t

In [22]:
class Command():
    '''
    Creates a command and an empty command list for UNIX command line programs/applications
    
    Attributes:
        command: Command to be performed on the command line
    '''
    
    def __init__(self):
        '''
        Init doc-string for Command class.
        '''
        pass; return None
    
    def init_cmd(self,command):
        '''
        Init command function for initializing commands to be used on UNIX command line.
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        return self.cmd_list

In [23]:
def split_4D(nii_4d,out_prefix,dim="-t",verbose=False):
    '''
    Wrapper function for FSL's fslsplit. Splits 4D nifti timeseries data and returns a list of 
    the split timeseries files.
    
    Arugments:
        nii_4d (nifti file): Input 4D file to be separated
        out_prefix (string): Output file prefix (no '.nii.gz', preferably to separate directory)
        dim (string): Dimension to separate along [default: '-t']
        verbose (bool, optional): Enable verbose output [default: False]
        
    Returns:
        nii_list (list): Sorted list of the zeropadded filenames of the split 4D timeseries.
    '''
    
    # Check dimension argument
    if dim != "-t" and dim != "-x" and dim != "-y" and dim != "-z":
        print("Unrecognized option for dimension. Using default of time.")
        dim = "-t"
        
    
    # Initialize command
    split = Command().init_cmd("fslsplit")
    
    # Add input/output arguments
    split.append(nii_4d)
    split.append(out_prefix)
    split.append(dim)
    
    if verbose:
        print(' '.join(split))
    
    # Perform/Execute command
    subprocess.call(split)
    
    # Create list of files
    nii_list = sorted(glob.glob(out_prefix + "*.nii*"))
    
    return nii_list

In [24]:
# n = split_4D("/Users/brac4g/Desktop/reg4D/test_data/filtered_func_data.nii.gz",
#         "/Users/brac4g/Desktop/reg4D/test1/split",
#         dim="-t")

In [25]:
def merge_4D(nii_list,out_prefix,dim="-tr",tr=2.000,verbose=False):
    '''
    Wrapper function for FSL's fslmerge. Merges a list of 3D nifti files and returns a merged file in some
    arbitrary dimension (t (time, with TR), x, y, or, z).
    
    Arugments:
        nii_list (list): List of the zeropadded filenames of the split 4D timeseries.
        out_prefix (string): Output file prefix (no '.nii.gz')
        dim (string): Dimension to separate along [default: '-tr']
        tr (float): Repetition Time (TR, in sec.) [default: 2.000]
        verbose (bool, optional): Enable verbose output [default: False]
        
    Returns:
        out_file (string): Merged output nifti file.
    '''
    
    # Check dimension argument
    if dim != "-tr" and dim != "-t" and dim != "-x" and dim != "-y" and dim != "-z":
        print("Unrecognized option for dimension. Using default of time with defined TR.")
        dim = "-tr"
        
    # Create empty file and path to retrieve
    out_prefix = File.make_filename_path(out_prefix)
    
    # Initialize command
    merge = Command().init_cmd("fslmerge")
    
    # Add input/output arguments
    merge.append(dim)
    merge.append(out_prefix)   
    merge.extend(nii_list)
    
    if dim == "-tr":
        merge.append(str(tr))
        
    if verbose:
        print(' '.join(merge))
    
    # Perform/Execute command
    subprocess.call(merge)
    
    out_file = out_prefix + ".nii.gz"
    
    return out_file

In [26]:
# merge_4D(nii_list,out_prefix,tr=3.000)

In [27]:
def apply_xfm_3D(nii_file,out_prefix,ref_vol,warp="",warp_app="relative",premat="",postmat="",
                mask="",interp="",padding_size="",use_qform=False,data_type="",super_sampling=False,
                super_level=2,verbose=False):
    '''
    Wrapper function for FSL's applywarp. Applies linear and non-linear transforms (xfms) in a one-step manner on
    3D images. Generally, this reduces image blurring as a result of reduced image resampling accross the application of 
    multiple transforms.
    
    Note: 
    - If wanting to apply more than two linear transforms, then the first two transformation matrices should be concatenated (via FSL's convert_xfm)
    - If wanting to apply more than one non-linear warp field, then the two warps should be combined (via FSL's convertwarp)
    
    Arugments:
        nii_file (nifti file): Input nifti file
        out_prefix (string): Output file (no '.nii.gz')
        ref_vol (nifti file): Reference volume
        warp (nifti file, optional): Warp file to reference
        warp_app (nifti file, optional): Warp field treatment. Valid options are: "relative" (default), and "absolute"
        premat (FSL mat file, optional): Pre-transform linear transformation matrix
        postmat (FSL mat file, optional): Post-transform linear transformation matrix
        mask (nifti file, optional): Mask, in reference space
        interp (string, optional): Interpolation method, options include: "nn","trilinear","sinc","spline"
        padding_size (int, optional): Extrapolates outside original volume by n voxels
        use_qform (bool, optional): Use s/qforms of ref_vol and nii_file images
        data_type (string,optional): Force output data type ["char" "short" "int" "float" "double"].
        super_sampling (bool, optional): Intermediary supersampling of output [default: False]
        super_level (int, optional): Level of intermediary supersampling, a for 'automatic' or integer level. [default: 2]
        verbose (bool, optional): Enable verbose output [default: False]
        
    Returns:
        out_file (string): Output file with the applied transform(s).
    '''
    
    # Create empty file and path to retrieve
    out_prefix = File.make_filename_path(out_prefix)
    
    # Initialize command
    xfm = Command().init_cmd("applywarp")
    
    # Add required input/output arguments
    xfm.append(f"--in={nii_file}")
    xfm.append(f"--out={out_prefix}")
    xfm.append(f"--ref={ref_vol}")
    
    # Add optional input arguments
    if warp:
        xfm.append(f"--warp={warp}")
        
    if warp_app == "relative":
        xfm.append("--rel")
    elif warp_app == "absolute":
        xfm.append("--abs")
        
    if premat:
        xfm.append(f"--premat={premat}")
        
    if postmat:
        xfm.append(f"--postmat={postmat}")
        
    if data_type:
        if data_type == "char" or data_type == "short" or data_type == "int" or data_type == "float" or data_type == "double":
            xfm.append(f"--datatype={data_type}")
        else:
            print(f"Unrecognized option: {data_type}. Using input data type.")
    
    if mask:
        xfm.append(f"--mask={mask}")
        
    if super_sampling:
        xfm.append("--super")
        if super_level:
            xfm.append(f"--superlevel={super_level}")
    
    if interp:
        if interp == "nn" or interp == "trilinear" or interp == "sinc" or interp == "spline":
            xfm.append(f"--interp={interp}")
    
    if padding_size:
        xfm.append(f"--paddingsize={padding_size}")
        
    if use_qform:
        xfm.append("--usesqform")
    
    if verbose:
        xfm.append("--verbose")
        print(' '.join(xfm))
    
    # Perform/Execute command
    subprocess.call(xfm)
    
    out_file = out_prefix + ".nii.gz"
    
    return out_file

In [28]:
# in_f="/Users/brac4g/Desktop/reg4D/test_data/example_func.nii.gz"
# out_f="/Users/brac4g/Desktop/reg4D/test_data/test_xfm"
# ref="/Users/brac4g/Desktop/reg4D/test_data/highres.nii.gz"
# premat="/Users/brac4g/Desktop/reg4D/test_data/example_func2highres.mat"

In [29]:
# apply_xfm_3D(in_f,out_f,ref,premat=premat,verbose=True)

In [30]:
def get_img_dims(nii_file):
    '''
    Reads and stores nifti image dimensions from input nifti file.
    
    Arugments:
        nii_file (nifti file): Input nifti file
        
    Returns:
        n_dim (int): N-dimensions of image (i.e. 2, for 2D, 3 for 3D, and 4 for 4D)
        x_dim (int): X-dimension length
        y_dim (int): Y-dimension length
        z_dim (int): Z-dimension length
        n_vol (int): Number of volumes in image (1 for 2D and 3D images, N>1 for 4D images) 
        x_vox (float): X-dimension voxel size
        y_vox (float): Y-dimension voxel size
        z_vox (float): Z-dimension voxel size
        tr (float): Repetition time from nifti header
    '''
    
    # Load image
    img = nib.load(nii_file)
    
    # Image dimensions
    n_dim = int(img.header['dim'][0])
    x_dim = int(img.header['dim'][1])
    y_dim = int(img.header['dim'][2])
    z_dim = int(img.header['dim'][3])
    n_vol = int(img.header['dim'][4])
    
    # Image voxel size
    x_vox = float(img.header['pixdim'][1])
    y_vox = float(img.header['pixdim'][2])
    z_vox = float(img.header['pixdim'][3])
    tr = float(img.header['pixdim'][4])
    
    return n_dim,x_dim,y_dim,z_dim,n_vol,x_vox,y_vox,z_vox,tr

In [189]:
# def img_res(nii_file,out_prefix,ref,resamp_vox=True,resamp_dim=True,verbose=False):
#     '''
#     Wrapper function for MIRTK's image resample-image binary. Performs image resampling of an image in referance 
#     to another image. This function is primarily intended for resampling a transformed image back to its 
#     original native space - implying that the reference (ref) image should be the original native space image.
    
#     Note: 
#     - Not all of MIRTK's resample-image options are present. This is to ensure consistent behavior.
#     - Only intended for 2D or 3D images. If the image is 4D, this function will exit as this will likely consume too much memory.
    
#     Arugments:
#         nii_file (nifti file): List of the zeropadded filenames of the split 4D timeseries.
#         out_prefix (string): Output file prefix (no '.nii.gz')
#         ref (nifti file): (Native space) Reference volume
#         resamp_vox (bool, optional): Resample voxel size to reference image [default: True]
#         resamp_dim (bool, optional): Resample image dimension to reference image [default: True]
#         verbose (bool, optional): Enable verbose output [default: False]
        
#     Returns:
#         out_file (string): Merged output nifti file.
#     '''
    
#     # Create empty file and path to retrieve
#     out_prefix = File.make_filename_path(out_prefix)
    
#     # Initialize command
#     rsm = Command().init_cmd("mirtk")
#     rsm.append("resample-image") # append mirtk sub-command
    
#     # Get image dimensions
#     [n_dim,x_dim,y_dim,z_dim,n_vol,x_vox,y_vox,z_vox,tr] = get_img_dims(nii_file=ref)
    
#     if n_dim > 3:
#         print(f"Image {nii_file} is greater than 3 dimensions. Exiting")
#         sys.exit(1)
    
#     # Add required input/output arguments
#     rsm.append(nii_file)
#     rsm.append(out_prefix + ".nii.gz")
    
#     if resamp_vox:
#         rsm.append("-size")
#         rsm.append(str(x_vox))
#         rsm.append(str(y_vox))
#         rsm.append(str(z_vox))
        
#     if resamp_dim:
#         rsm.append("-imsize")
#         rsm.append(str(x_dim))
#         rsm.append(str(y_dim))
#         rsm.append(str(z_dim))
        
#     if verbose:
#         rsm.append("-verbose")
#         print(' '.join(rsm))
        
#     # Perform/Execute command
#     subprocess.call(rsm)
    
#     out_file = out_prefix + ".nii.gz"
    
#     return out_file

In [ ]:
# def img_res(nii_file, out_prefix, ref, resamp_vox=True, resamp_dim=True, verbose=False):
#     '''
#     Wrapper function for MIRTK's image resample-image binary. Performs image resampling of an image in referance
#     to another image. This function is primarily intended for resampling a transformed image back to its
#     original native space - implying that the reference (ref) image should be the original native space image.

#     Note:
#     - Not all of MIRTK's resample-image options are present. This is to ensure consistent behavior.
#     - Only intended for 2D or 3D images. If the image is 4D, this function will exit as this will likely consume too much memory.

#     Arugments:
#         nii_file (nifti file): List of the zeropadded filenames of the split 4D timeseries.
#         out_prefix (string): Output file prefix (no '.nii.gz')
#         ref (nifti file): (Native space) Reference volume
#         resamp_vox (bool, optional): Resample voxel size to reference image [default: True]
#         resamp_dim (bool, optional): Resample image dimension to reference image [default: True]
#         verbose (bool, optional): Enable verbose output [default: False]

#     Returns:
#         out_file (string): Merged output nifti file.
#     '''

#     # Create empty file and path to retrieve
#     out_prefix = File.make_filename_path(out_prefix)

#     # Initialize command
#     rsm = Command().init_cmd("mirtk")
#     rsm.append("resample-image")  # append mirtk sub-command

#     # Get image dimensions
#     [n_dim, x_dim, y_dim, z_dim, n_vol, x_vox, y_vox, z_vox, tr] = get_img_dims(nii_file=ref)

#     if n_dim > 3:
#         print(f"Image {nii_file} is greater than 3 dimensions. Exiting")
#         sys.exit(1)

#     # Add required input/output arguments
#     rsm.append(nii_file)
#     out_prefix = out_prefix + ".nii.gz"
#     rsm.append(out_prefix)

#     if resamp_vox:
#         rsm.append("-size")
#         rsm.append(str(x_vox))
#         rsm.append(str(y_vox))
#         rsm.append(str(z_vox))

#     if resamp_dim:
#         rsm.append("-imsize")
#         rsm.append(str(x_dim))
#         rsm.append(str(y_dim))
#         rsm.append(str(z_dim))

#     if verbose:
#         rsm.append("-verbose")
#         print(' '.join(rsm))

#     # Perform/Execute command
#     subprocess.call(rsm)

#     # out_file = out_prefix + ".nii.gz"
#     out_file = out_prefix

#     return out_file

In [31]:
def img_res(nii_file, out_prefix, ref, resamp_vox=True, resamp_dim=True, verbose=False):
    '''
    Wrapper function for MIRTK's image resample-image binary. Performs image resampling of an image in referance
    to another image. This function is primarily intended for resampling a transformed image back to its
    original native space - implying that the reference (ref) image should be the original native space image.

    Note:
    - Not all of MIRTK's resample-image options are present. This is to ensure consistent behavior.
    - Only intended for 2D or 3D images. If the image is 4D, this function will exit as this will likely consume too much memory.

    Arugments:
        nii_file (nifti file): List of the zeropadded filenames of the split 4D timeseries.
        out_prefix (string): Output file prefix (no '.nii.gz')
        ref (nifti file): (Native space) Reference volume
        resamp_vox (bool, optional): Resample voxel size to reference image [default: True]
        resamp_dim (bool, optional): Resample image dimension to reference image [default: True]
        verbose (bool, optional): Enable verbose output [default: False]

    Returns:
        out_file (string): Merged output nifti file.
    '''

    # Create empty file and path to retrieve
    out_prefix = File.make_filename_path(out_prefix)

    # Initialize command
    rsm = Command().init_cmd("mirtk")
    rsm.append("resample-image")  # append mirtk sub-command

    # Get image dimensions
    [n_dim, x_dim, y_dim, z_dim, n_vol, x_vox, y_vox, z_vox, tr] = get_img_dims(nii_file=ref)

    if n_dim > 3:
        print(f"Image {nii_file} is greater than 3 dimensions. Exiting")
        sys.exit(1)

    # Add required input/output arguments
    if not nii_file in '.nii.gz':
        # print(True)
        nii_file = nii_file + '.nii.gz'
    rsm.append(nii_file)
    # out_prefix = out_prefix + ".nii.gz"
    rsm.append(out_prefix)

    if resamp_vox:
        rsm.append("-size")
        rsm.append(str(x_vox))
        rsm.append(str(y_vox))
        rsm.append(str(z_vox))

    if resamp_dim:
        rsm.append("-imsize")
        rsm.append(str(x_dim))
        rsm.append(str(y_dim))
        rsm.append(str(z_dim))

    if verbose:
        rsm.append("-verbose")
        print(' '.join(rsm))

    # Perform/Execute command
    subprocess.call(rsm)

    # out_file = out_prefix + ".nii.gz"
    out_file = out_prefix

    return out_file

In [33]:
f="/Users/brac4g/Desktop/reg4D/tmp_dir6550/tmp_dir4617/func_xfm_0279"

In [35]:
if not f in '.nii.gz':
    print(True)

True


In [ ]:
img_res(nii_file, out_prefix, ref, resamp_vox=True, resamp_dim=True, verbose=False)

In [187]:
in_f = "/Users/brac4g/Desktop/reg4D/test_data/test_xfm.nii.gz"
out_f = "/Users/brac4g/Desktop/reg4D/test_data/func_native_aligned"
ref="/Users/brac4g/Desktop/reg4D/test_data/example_func.nii.gz"

In [190]:
img_res(in_f,out_f,ref,verbose=True)

mirtk resample-image /Users/brac4g/Desktop/reg4D/test_data/test_xfm.nii.gz /Users/brac4g/Desktop/reg4D/test_data/func_native_aligned -size 2.799999952316284 2.799999952316284 3.0 -imsize 80 80 40 -verbose


'/Users/brac4g/Desktop/reg4D/test_data/func_native_aligned.nii.gz'

In [ ]:
def apply_xfm_4D(nii_file,out_prefix,ref_vol,warp="",warp_app="relative",premat="",postmat="",
                mask="",interp="",padding_size="",use_qform=False,data_type="",super_sampling=False,
                super_level=2,verbose=False):
    '''working doc-string'''
    
    # random number for random number generator
    n = 10000 # maximum N
    
    out_tmp_1 = os.path.join(os.path.basename(out_prefix), 'tmp_dir' + + str(random.randint(0, n)))
    out_name_1 = os.path.join(out_tmp_1,'func_split')
    
    if not os.path.exists(out_tmp_1):
        os.makedirs(out_tmp_1)
        
    

In [15]:
def apply_xfm_4D(nii_file,out_prefix,ref_xfm,ref_target,num_jobs="infer",dim="-tr",
  tr="infer",warp="",warp_app="relative",premat="",postmat="",resamp_vox=True,
  resamp_dim=False,mask="",interp="",padding_size="",use_qform=False,data_type="",
  super_sampling=False,super_level=2,verbose=False):
    '''working doc-string'''
    
    # random number for random number generator
    n = 10000 # maximum N
    
    # Get image info
    [n_dim,x_dim,y_dim,z_dim,n_vol,x_vox,y_vox,z_vox,tr_img] = get_img_dims(nii_file=nii_file)
    
    # Check split dimension variable
    if dim == "-tr":
        split_dim = "-t"
    else:
        split_dim = dim
        
    # Get number of jobs for parallelization
    if num_jobs == 'infer':
        num_jobs = multiprocessing.cpu_count()
        
    # Get TR
    if tr == 'infer':
        tr = tr_img
        
    # Make keyword argument dictionaries
    # Command 1 dictionary
    cmd_dict_1 = {"ref_vol":ref_xfm} # Required argument(s) not in command list

    if warp:
        tmp_dict = {"warp":warp}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"warp":""}
        cmd_dict_1.update(tmp_dict)

    if warp_app:
        tmp_dict = {"warp_app":f"{warp_app}"}
        cmd_dict_1.update(tmp_dict)

    if premat:
        tmp_dict = {"premat":premat}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"premat":""}
        cmd_dict_1.update(tmp_dict)

    if postmat:
        tmp_dict = {"postmat":postmat}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"postmat":""}
        cmd_dict_1.update(tmp_dict)

    if mask:
        tmp_dict = {"mask":mask}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"mask":""}
        cmd_dict_1.update(tmp_dict)

    if interp:
        tmp_dict = {"interp":interp}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"interp":""}
        cmd_dict_1.update(tmp_dict)

    if padding_size:
        tmp_dict = {"padding_size":padding_size}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"padding_size":""}
        cmd_dict_1.update(tmp_dict)

    if use_qform:
        tmp_dict = {"use_qform":use_qform}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"use_qform":False}
        cmd_dict_1.update(tmp_dict)

    if data_type:
        tmp_dict = {"data_type":data_type}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"data_type":""}
        cmd_dict_1.update(tmp_dict)

    if super_sampling:
        tmp_dict = {"super_sampling":super_sampling}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"super_sampling":False}
        cmd_dict_1.update(tmp_dict)

    if super_level:
        tmp_dict = {"super_level":super_level}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"super_level":2}
        cmd_dict_1.update(tmp_dict)

    if verbose:
        tmp_dict = {"verbose":verbose}
        cmd_dict_1.update(tmp_dict)
    else:
        tmp_dict = {"verbose":False}
        cmd_dict_1.update(tmp_dict)

    # Command 2 dictionary
    cmd_dict_2 = {"ref":ref_target} # Required argument(s) not in command list

    if not resamp_vox and not resamp_dim:
        cp_dir = True
    else:
        cp_dir = False
        if resamp_vox:
            tmp_dict = {"resamp_vox":resamp_vox}
            cmd_dict_2.update(tmp_dict)
        else:
            tmp_dict = {"resamp_vox":False}
            cmd_dict_2.update(tmp_dict)

        if resamp_dim:
            tmp_dict = {"resamp_dim":resamp_dim}
            cmd_dict_2.update(tmp_dict)
        else:
            tmp_dict = {"resamp_dim":False}
            cmd_dict_2.update(tmp_dict)

        if verbose:
            tmp_dict = {"verbose":verbose}
            cmd_dict_2.update(tmp_dict)
        else:
            tmp_dict = {"verbose":False}
            cmd_dict_2.update(tmp_dict)
    
    # Make temporay directories
    out_tmp = os.path.join(os.path.basename(out_prefix), 'tmp_dir' + str(random.randint(0, n)))
    
    out_tmp_1 = os.path.join(os.path.basename(out_tmp), 'tmp_dir' + str(random.randint(0, n)))
    out_name_1 = os.path.join(out_tmp_1,'func_split')

    out_tmp_2 = os.path.join(os.path.basename(out_tmp), 'tmp_dir' + str(random.randint(0, n)))
    out_name_2 = os.path.join(out_tmp_2,'func_xfm')

    out_tmp_3 = os.path.join(os.path.basename(out_tmp), 'tmp_dir' + str(random.randint(0, n)))
    out_name_3 = os.path.join(out_tmp_3,'func_rsm')
    
    if not os.path.exists(out_tmp_1):
        os.makedirs(out_tmp_1)

    if not os.path.exists(out_tmp_2):
        os.makedirs(out_tmp_2)

    if not os.path.exists(out_tmp_3):
        os.makedirs(out_tmp_3)
        
    pre_xfm_list = split_4D(nii_4d=nii_file,out_prefix=out_name_1,dim=split_dim,verbose=verbose)

    ###### parallel starts here ######
    
    out_tmp_names_list_2 = make_out_list(out_name=out_name_2,max_val=n_vol)
    out_tmp_names_list_3 = make_out_list(out_name=out_name_3,max_val=n_vol)
    
    cmd_tup_2 = multiproc_cmd_list_tuple(in_list=pre_xfm_list, out_list=out_tmp_names_list_2, **cmd_dict_1)
    cmd_tup_3 = multiproc_cmd_list_tuple(in_list=out_tmp_names_list_2, out_list=out_tmp_names_list_3, **cmd_dict_2)
    
    with multiprocessing.Pool(processes=num_jobs) as pool:
        proc_1 = pool.starmap(apply_xfm_3D, cmd_tup_2)
        
    with multiprocessing.Pool(processes=num_jobs) as pool:
        proc_2 = pool.starmap(img_res, cmd_tup_3)

    # Do stuff here

    ###### parallel ends here ######

    xfm_list = sorted(glob.glob(out_name_3 + "*.nii*"))

    out_file = merge_4D(nii_list=xfm_list,out_prefix=out_prefix,dim=dim,tr=tr,verbose=False)
    
    return out_file

In [6]:
def multiproc_cmd_list_tuple(in_list, out_list, **kwargs):
    '''
    Creates a tuple of lists that contains the input arguments in the format that multiprocessing's Pool uses.
    This function is intended for use with multiprocessing's Pool class and reg4D's apply_xfm_4D function.
    
    Arguments:
        in_list (list): Input list of filenames
        out_list (list): Output list of filenames
        **kwargs (string,dict, key,value pairs): key=value pairs - Most useful when function arguments are passed as a dictionary
    
    Returns:
        cmd_tup (tuple): Tuple of (command) lists that can be used as input to  multiprocessing's Pool class.
    '''

    # Empty list
    cmd_list = list()

    # for idx,file in enumerate(in_list):
    for idx in range(0, len(in_list), 1):
        tmp_list = [in_list[idx], out_list[idx]]
        for key, item in kwargs.items():
            tmp_list.append(f"{item}")
        cmd_list.append(tmp_list)

    cmd_tup = tuple(cmd_list)
    return cmd_tup

In [10]:
n_vol = 10

In [7]:
word = "word"

In [8]:
name_list = list()

In [12]:
for idx in range(0,n_vol,1):
    tmp_num = '{:04}'.format(int(idx))
    tmp_name = word + f"_{tmp_num}"
    name_list.append(tmp_name)
    
print(name_list)

['word_0000', 'word_0001', 'word_0002', 'word_0003', 'word_0004', 'word_0005', 'word_0006', 'word_0007', 'word_0008', 'word_0009']


In [13]:
def make_out_list(out_name, max_val):
    '''
    Creates an output name list provided an output name and a maximum value (not inclusive) to iterate to.
    
    Arguments:
        out_name (string): Output name prefix
        max_val (int): Maximum value (not inclusive) to iterate to
    
    Returns:
        name_list (list): Output names list with zeropadded numbers added to the end.
    '''
    
    name_list = list()
    
    for idx in range(0,max_val,1):
        tmp_num = '{:04}'.format(int(idx))
        tmp_name = out_name + f"_{tmp_num}"
        name_list.append(tmp_name)
        
    return name_list

In [36]:
# def file_parts(file):
#     '''
#     Divides file with file path into: path, filename, extension.
    
#     Arguments:
#         file (string): File with absolute filepath
        
#     Returns: 
#         path (string): Path of input file
#         filename (string): Filename of input file, without the extension
#         ext (string): Extension of input file
#     '''
    
#     [path, file_with_ext] = os.path.split(file)
    
#     # Make condition for gzipped files
#     if '.gz' in file_with_ext:
#         file_with_ext = file_with_ext[:-3]
#         [filename,ext] = os.path.splitext(file_with_ext)
#         ext = ext + '.gz'
#     else:
#         [filename,ext] = os.path.splitext(file_with_ext)
    
#     path = str(path)
#     filename = str(filename)
#     ext = str(ext)
    
#     return path,filename,ext

In [38]:
def cp_img(file,dest_name):
    '''
    Copies a gzipped nifti image file. Primarily intended for copying single file image data.
    
    Arguments:
        file (nifit file): File path to source (image) file
        dest_name (string): Absolute path output file name.
        
    Returns:
        out_file (string): Absolute path to output file.
    '''
    
    if not file in '.nii.gz':
        file = file + '.nii.gz'
        
    if not dest_name in '.nii.gz':
        out_file = dest_name + '.nii.gz'
    
    shutil.copy(file,out_file)
    
    return out_file

In [39]:
def cp_dir_imgs(in_list,out_list,max_val):
    '''
    Copies a directory of image data provided an input, and output list of filenames in addition to a maximum 
    value (not inclusive) to iterate to.
    
    No files are returned, as they are copied to some output directory.
    
    Arguments:
        in_list (list): Input name list
        out_list (list): Output name list
        max_val (int): Maximum value (not inclusive) to iterate to
    
    Returns:
        None
    '''
    
    for idx in range(0,max_val,1):
        in_file = in_list[idx]
        out_file = out_list[idx]
        cp_img(file=in_file,dest_name=out_file)